# Wikipedia Corpus

Corpus from: https://dumps.wikimedia.org/dewiki/20200820/

Sentences for comparison from: https://github.com/t-systems-on-site-services-gmbh/german-wikipedia-text-corpus

In [1]:
#imports
from xml.etree.ElementTree import *
import xml.etree.ElementTree as ET
from collections import Counter
import os
import pprint
import gensim
from gensim import corpora
from gensim import models
from gensim import similarities
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models import LdaMulticore
import nltk
from nltk.corpus import stopwords
from smart_open import open 
import spacy
import de_core_news_md

In [2]:
number_objects = 200
number_corpus_object = 50

## Preprocessing:

1. Lade das Language-Modell "de_core_news_xx" von Spacy
2. Erstelle die Funktion "preprocess_text in die das Object "text" verarbeiet wird
3. Aufrufen von "text" mit dem Language-Modell. Das Modell wandelt "text" in Tokens um -> "prep_text"
4. In "prep-tokens" werden durch eine Schleife, alle Tokens von "prep_text" gespeichert
    4.1 Entferne Stopwörter in Tokens 
    4.2 Lemmatisiere diese
    4.3 Schreibe alles klein
5. Zurückgegeben werden alle Tokens, ohne Stopwörter, in lemma Form, klein geschrieben des Inputs "text"

In [3]:
# load the language model from spacy
spacy_data = de_core_news_md.load()

def preprocess_text(text):
    # load and tokenize text
    prep_text = spacy_data(text)
    # list for tokens
    prep_tokens = []
    # for every token in text
    for token in prep_text:
        # remove stopwords and punctuatiuon
        if token.pos_ != 'PUNCT' and token.is_stop == False:
            # lemmatize and transform to lowercase
            lemma_token = token.lemma_.lower()
            # remove non-alphabetic tokens
            if lemma_token.isalpha() or lemma_token == '-PRON-':
                prep_tokens.append(lemma_token)
    # return preprocessed text 
    return prep_tokens

## Build the corpus

Create a corpus from the text contents of the XML file:

First test:

Print text from \<text>:

In [4]:
xml_file = "/Volumes/MAINZ_BB/dewiki-20200820-pages-articles-multistream.xml"

Object-Stream Test

1. Erstelle Klasse "MyCorpus"
2. Funktion "_iter_" nimmt den Imput "self" als Generator
3. Schleife durchläuft alle Elemente der XML-Datei. Events als "start" und "end" darstellen.
    3.1 Das Document wird dargestellt als Text zwischen "text" Tags der XML-Datei
    3.2 Gib jedes gefundene Objekt an Generatror zurück 
    3.3 Lösche die das verarbeitete Objekt

Now define the corpus:

1. Klasse "Mycorpus_small"
2. Funktion "_iter_" nimmt den Imput "self" als Generator
3. Schleife durchläuft alle Elemente der XML-Datei. Events als "start" und "end" darstellen.
    3.1 Das Document wird dargestellt als Text zwischen "text" Tags der XML-Datei
    3.2 verarbeite nur die ersten 200 Objekte
    3.3 Gib jedes gefundene Objekt an Generatror zurück 
    3.4 Zähl Index hoch
    3.5 Lösche die das verarbeitete Objekt

In [5]:
# Define a smaller corpus, containing only the first 200 documents:
class MyCorpus_small:
    def __iter__(self):
        index = 0
        # define the XML tree
        for event, elem in ET.iterparse(xml_file, events = ("start", "end")):
            if index < number_corpus_object:
                # Each document is represented as an object between <text> tags in the xml file
                if event == 'end' and "text" in elem.tag:
                    # Transfom the corpus to vectors
                    yield dictionary.doc2bow(preprocess_text(elem.text))
                    index+=1
                    elem.clear()
            else:
                break

In [6]:
corpus_small = MyCorpus_small()

---

Get texts from the XML File
Einsellen wie viele Artikel geladen werden sollen

In [7]:
%%time
text_ids = {}
texts = []
index = 0
for event, elem in ET.iterparse(xml_file, events = ("start", "end")):        
    if index < number_objects:
        if event == 'end' and "text" in elem.tag:
            text_ids[index]=str(elem.text)
            index += 1  
            texts.append(str(elem.text))
            elem.clear()
    else:
        break


CPU times: user 52.6 ms, sys: 2.86 ms, total: 55.4 ms
Wall time: 64.7 ms


---

## Build the Dictionary

def build_dictionary(xml_file):
    index = 0
    first_elem = True
    for event, elem in ET.iterparse(xml_file, events = ("start", "end")):        
        if index < number_objects:
            if event == "end" and "text" in elem.tag:
                text = preprocess_text(elem.text)
                if first_elem:
                    dictionary = Dictionary([text])
                    first_elem = False
                    index += 1
                else:
                    dictionary.add_documents([text])
                    index += 1
                elem.clear()
        else:
            break
    return dictionary

In [8]:
#load the dictionary
dictionary = Dictionary.load('data/wiki.dict')

In [9]:
print(dictionary)

Dictionary(20309 unique tokens: ['abc', 'abkehr', 'ablehnen', 'abrufen', 'abschluss']...)


---

## Similarity with LDA (Latent Dirichlet Allocation)

### Train the LDA model

Parameters:
* corpus: the corpus
* num_topics: topics to be extracted from the training corpus
* id2word: id to word mapping, the dictionary
* workers: number of cpu cores used

Currently not working with the streamed corpus. test_corpus is a temporary solution (hopefully) which contains the first 200 documents, manually added to a list.

In [10]:
corpus_small_vectors = []
for vector in corpus_small:
        corpus_small_vectors.append(vector)

First experiments have shown that a topic number of 10 (default) is too low. 100 resulted in better disctinction between the different articles.
__Further fine tuning needed here__

In [11]:
#load the dictionary
lda = LdaModel.load("data/index_wiki.txt")

In [19]:
%%time
corpus_index = similarities.MatrixSimilarity(lda[corpus_small_vectors], num_features=len(dictionary))

CPU times: user 426 ms, sys: 841 ms, total: 1.27 s
Wall time: 238 ms


In [63]:
test_doc_raw = texts[2]
test_vec = dictionary.doc2bow(preprocess_text(test_doc_raw))
#print(test_vec)
# convert to lda space
test_vec_lda = lda[test_vec]
#print(test_vec_lda)

In [64]:
sims = corpus_index[test_vec_lda]
plagiarism_id = max(list(enumerate(sims)), key=lambda x:x[1])[0]
plagiarism_weight = max(list(enumerate(sims)), key=lambda x:x[1])[1]

<h2><b>Ergebnis der Plagiatsprüfung</b></h2>

In [65]:
if plagiarism_weight >= 0.75:
    print("Plagiat gefunden: ")
    print(texts[plagiarism_id])
else:
    print("Kein Plagiat gefunden")

Plagiat gefunden: 
[[Datei:Ang Lee - 66eme Festival de Venise (Mostra) 2.jpg|mini|hochkant|Ang Lee, 2009]]
'''Ang Lee''' ({{zh|c=李安|p=Lǐ Ān}}; * [[23. Oktober]] [[1954]] in [[Chaozhou (Pingtung)|Chaozhou]],<ref>{{Internetquelle |url=http://www.taipeitimes.com/News/taiwan/archives/2006/03/07/2003296130 |titel=Family and friends praise Ang Lee's quiet dedication – Taipei Times |abruf=2018-09-17}}</ref> [[Landkreis Pingtung]], [[Republik China (Taiwan)]]) ist ein taiwanischer [[Filmregisseur]], [[Drehbuchautor]] und Produzent.<ref name="GreenCard">{{Internetquelle|url=https://www.youtube.com/watch?v=7p9vRrzQ7-U|titel=2016.11.12中天的夢想驛站完整版　戰爭人性與電影科技　李安：視覺對我是信仰|werk=Chung T’ien Television|datum=2016-11-12|abruf=2019-12-02|sprache=zh}}</ref> Er ist als vielfach ausgezeichneter Regisseur bekannt für so unterschiedliche Filme wie ''[[Eat Drink Man Woman]]'', die Jane-Austen-Adaption ''[[Sinn und Sinnlichkeit (1995)|Sinn und Sinnlichkeit]]'' und den Martial Arts-Film ''[[Tiger and Dragon]]''. Fü